In [1]:
library(rstan)

Warning message:
"package 'rstan' was built under R version 4.0.5"
Loading required package: StanHeaders

Warning message:
"package 'StanHeaders' was built under R version 4.0.5"
Loading required package: ggplot2

rstan (Version 2.21.2, GitRev: 2e1f913d3ca3)

For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)

Do not specify '-march=native' in 'LOCAL_CPPFLAGS' or a Makevars file



In [2]:
modelString = "
    data {
        int<lower=0> N ;
        int y[N] ; // y is a length-N vector of integers
    }
    parameters {
        real<lower=0,upper=1> theta ;
    }
    model {
        theta ~ beta(1,1) ;
        y ~ bernoulli(theta) ;
    }
" # close quote for modelString
## make sure to retype "~"

In [3]:
# stanDso = stan_model(model_code=modelString, verbose = FALSE)
# ?stan_model
##note: I added a line from https://stackoverflow.com/questions/56256884/error-on-the-stan-file-compilation-using-r-3-6-0-and-win-10/56328910#56328910 to .REnviron

In [ ]:
# Create some fictitious data:
N = 50 ; z = 10 ; y = c(rep(1,z),rep(0,N-z))
dataList = list( y = y , N = N )
stanFit = sampling( object=stanDso , data=dataList ,
chains=3 , iter=1000 , warmup=200 , thin=1 )
dataList

In [ ]:
# Load rjags, coda, and DBDA2E functions:
source("DBDA2E-utilities.R")
# Convert stan format to coda format:
mcmcCoda = mcmc.list( lapply( 1:ncol(stanFit) ,
function(x) { mcmc(as.array(stanFit)[,x,]) } ) )
# Graph chain diagnostics using DBDA2E function:
diagMCMC( mcmcCoda , parName=c("theta") )

### General structure of Stan (complex) model specification 

In [ ]:
data {
... declarations ...
}
transformed data {
... declarations ... statements ...
}
parameters {
... declarations ...
}
transformed parameters {
... declarations ... statements ...
}
model {
... declarations ... statements ...
}
generated quantities {
... declarations ... statements ...
}

### Sampling the prior in Stan

In [ ]:
modelString = "
data {
int<lower=0> N ;
int y[N] ;
}
parameters {
real<lower=0,upper=1> theta ;
}
model {
theta ˜ beta(1,1) ;
y ˜ bernoulli(theta) ;
}
" # close quote for modelString

In [ ]:
stanDso = stan_model(model_code=modelString)

### simplified scripts for frequently used analyses

In [6]:
# Load The data
myData = read.csv("https://raw.githubusercontent.com/boboppie/kruschke-doing_bayesian_data_analysis/master/2e/z15N50.csv") #switched to url 
# Load the functions genMCMC, smryMCMC, and plotMCMC:
source("Stan-Ydich-Xnom1subj-MbernBeta.R")
# Specify filename root and graphical format for saving output.
fileNameRoot = "Stan-Ydich-Xnom1subj-MbernBeta-"
graphFileType = "eps" # or "png" or "pdf" etc.
# Generate the MCMC chain:
mcmcCoda = genMCMC( data=myData , numSavedSteps=10000 , saveName=fileNameRoot )
# Display diagnostics of chain, for specified parameters:
parameterNames = varnames(mcmcCoda) # get all parameter names
for ( parName in parameterNames ) {
diagMCMC( mcmcCoda , parName=parName ,
saveName=fileNameRoot , saveType=graphFileType )
}
# Get summary statistics of chain:
summaryInfo = smryMCMC( mcmcCoda , compVal=0.5 , rope=c(0.45,0.55) ,
saveName=fileNameRoot )
# Display posterior information:
plotMCMC( mcmcCoda , data=myData , # compVal=0.5 , rope=c(0.45,0.55) ,
saveName=fileNameRoot , saveType=graphFileType )
# Use Stan display functions instead of DBDA2E functions:
# Load the stanFit object that was saved by genMCMC:
load("Stan-Ydich-Xnom1subj-MbernBeta-StanFit.Rdata")
# Display information:
show(stanFit)
openGraph()
traceplot(stanFit,pars=c("theta"))
openGraph()
plot(stanFit,pars=c("theta"))

Warning message in file(filename, "r", encoding = encoding):
"cannot open file 'Stan-Ydich-Xnom1subj-MbernBeta.R': No such file or directory"


ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection
